# 功能：识别对勾字段，判断是否毕业

In [1]:
# coding:utf-8
import cv2
import numpy as np
import os
import pandas as pd
dir_from = '../data/cut_image/'
key = ['high_school_pass', 'junior_college_pass',
       'undergraduate_pass', 'graduate_pass']
file_name = []

In [2]:
df = pd.DataFrame(columns=['登记表编号','高中是否毕业','大专是否毕业','本科是否毕业','研究生是否毕业','是否毕业（其它）'])
df

,登记表编号,高中是否毕业,大专是否毕业,本科是否毕业,研究生是否毕业,是否毕业（其它）


In [3]:
file_id = []
high_school = []
junior_college = []
undergraduate = []
graduate = []
for di in range(len(key)):
    file_dir = dir_from + key[di] + '/'
    for root, dirs, files in os.walk(file_dir):
        file_name = files
    for fi in file_name:
        img1 = cv2.imread('{}{}'.format(file_dir, fi))
        print("read {}{}".format(file_dir, fi))
        if (di == 0):
            file_id.append(fi.split(sep='.')[0])
        gray_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        thresh_img = cv2.adaptiveThreshold(
            ~gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -2)

        h_img = thresh_img.copy()
        v_img = thresh_img.copy()
        h_structure = cv2.getStructuringElement(
            cv2.MORPH_RECT, (15, 1))  # 形态学因子
        h_erode_img = cv2.erode(h_img, h_structure, 1)
        h_dilate_img = cv2.dilate(h_erode_img, h_structure, 1)

        v_structure = cv2.getStructuringElement(
            cv2.MORPH_RECT, (1, 15))  # 形态学因子
        v_erode_img = cv2.erode(v_img, v_structure, 1)
        # 膨胀图像
        v_dilate_img = cv2.dilate(v_erode_img, v_structure, 1)
        mask_img = h_dilate_img+v_dilate_img  # 纵横线交叉
        joints_img = cv2.bitwise_and(h_dilate_img, v_dilate_img)  # 交点图
        final_img = mask_img ^ thresh_img
        if (len(joints_img.nonzero()[0]) == 0):
            flag = 'unknown'
            value = '无'
            len1 = len2 = 0
        else:
            a = np.nonzero(joints_img)  # 找最靠左上角的交点做参照点
            t = (a[0]-10)**2+(a[1]-25)**2
            min_loc = t.argmin()
            x = a[1][min_loc]
            y = a[0][min_loc]
            cropImg11 = final_img[(y+4):(y+13), (x+4):(x+13)]  # 格子中
            cropImg12 = final_img[(y-7):(y-2), (x-4):(x+15)]  # 格子上
            cropImg13 = final_img[(y+17):(y+23), (x-4):(x+15)]  # 格子下
            len1 = len(cropImg11.nonzero()[
                       0])+len(cropImg12.nonzero()[0])+len(cropImg13.nonzero()[0])
            print(x, y, len1)

            t = (a[0]-10)**2+(a[1]-img1.shape[1]+25)**2
            min_loc = t.argmin()
            x2 = a[1][min_loc]-15
            y2 = a[0][min_loc]
            cropImg21 = final_img[(y2+4):(y2+13), (x2+4):(x2+13)]  # 格子中
            cropImg22 = final_img[(y2-7):(y2-2), (x2-4):(x2+15)]  # 格子上
            cropImg23 = final_img[(y2+17):(y2+23), (x2-4):(x2+15)]  # 格子下
            len2 = len(cropImg21.nonzero()[
                       0])+len(cropImg22.nonzero()[0])+len(cropImg23.nonzero()[0])
            print(x2, y2, len2)

            if((len1 > 15) & (len2 < 15)):
                flag = 'yes'
                value = '是'
            elif((len1 < 15) & (len2 > 15)):
                flag = 'no'
                value = '否'
            else:
                flag = 'unknown'
                value = '无'
        if(di == 0):
            high_school.append(value)
        elif(di == 1):
            junior_college.append(value)
        elif(di == 2):
            undergraduate.append(value)
        else:
            graduate.append(value)
        '''
        folder = os.path.exists('../data/pass_or_not/{}'.format(key[di]))
        if not folder:
            os.makedirs('../data/pass_or_not/{}'.format(key[di]))
        cv2.imwrite('../data/pass_or_not/{}/{}_({},{})_{}'.format(key[di],
                                                                  flag, len1, len2, fi), img1)
        '''

read ../data/cut_image/high_school_pass/20110007.jpg
52 13 11
92 13 0
read ../data/cut_image/high_school_pass/20110190.jpg
52 12 0
93 11 9
read ../data/cut_image/high_school_pass/20110133.jpg
52 13 33
92 13 0
read ../data/cut_image/high_school_pass/20110230.jpg
52 13 0
93 13 0
read ../data/cut_image/high_school_pass/20110335.jpg
29 13 38
89 12 0
read ../data/cut_image/high_school_pass/20110155.jpg
52 13 0
92 13 0
read ../data/cut_image/high_school_pass/20140315.jpg
43 16 31
103 16 0
read ../data/cut_image/high_school_pass/20110052.jpg
52 12 38
92 12 0
read ../data/cut_image/high_school_pass/20110062.jpg
52 13 0
92 13 0
read ../data/cut_image/high_school_pass/20140330.jpg
42 17 0
103 16 0
read ../data/cut_image/high_school_pass/20110013.jpg
51 12 0
91 12 0
read ../data/cut_image/high_school_pass/20110301.jpg
52 12 0
91 12 0
read ../data/cut_image/high_school_pass/20110144.jpg
52 13 0
92 13 0
read ../data/cut_image/high_school_pass/20110354.jpg
30 14 0
90 13 0
read ../data/cut_image/high

42 16 37
102 16 1
read ../data/cut_image/high_school_pass/201100212.jpg
56 13 37
95 13 0
read ../data/cut_image/high_school_pass/20110101.jpg
51 13 38
91 13 20
read ../data/cut_image/high_school_pass/201100171.jpg
57 14 36
97 14 0
read ../data/cut_image/high_school_pass/201100259.jpg
27 13 37
86 13 0
read ../data/cut_image/high_school_pass/20110340.jpg
30 13 0
90 13 0
read ../data/cut_image/high_school_pass/20110351.jpg
31 13 0
90 13 0
read ../data/cut_image/high_school_pass/20110225.jpg
53 14 0
94 13 2
read ../data/cut_image/high_school_pass/20140321.jpg
42 17 0
103 16 0
read ../data/cut_image/high_school_pass/20110074.jpg
51 13 0
91 13 41
read ../data/cut_image/high_school_pass/20110161.jpg
52 14 35
92 14 0
read ../data/cut_image/high_school_pass/201100016.jpg
52 13 3
91 13 1
read ../data/cut_image/high_school_pass/20110221.jpg
52 14 0
94 15 0
read ../data/cut_image/high_school_pass/20110294.jpg
51 13 4
91 12 0
read ../data/cut_image/high_school_pass/20110096.jpg
52 13 37
92 13 0
rea

read ../data/cut_image/junior_college_pass/20140322.jpg
29 19 0
90 19 0
read ../data/cut_image/junior_college_pass/20110270.jpg
51 13 0
91 13 2
read ../data/cut_image/junior_college_pass/201100210.jpg
26 13 0
86 13 0
read ../data/cut_image/junior_college_pass/20110080.jpg
51 13 11
91 13 0
read ../data/cut_image/junior_college_pass/20110255.jpg
53 13 0
94 13 6
read ../data/cut_image/junior_college_pass/20140156.jpg
41 16 51
101 16 0
read ../data/cut_image/junior_college_pass/20110310.jpg
55 13 30
94 13 0
read ../data/cut_image/junior_college_pass/20110215.jpg
53 13 0
93 13 0
read ../data/cut_image/junior_college_pass/20110228.jpg
54 13 35
95 13 5
read ../data/cut_image/junior_college_pass/201100223.jpg
27 13 0
87 12 0
read ../data/cut_image/junior_college_pass/201100285.jpg
43 15 32
102 14 0
read ../data/cut_image/junior_college_pass/20110403.jpg
31 13 2
91 14 8
read ../data/cut_image/junior_college_pass/20110032.jpg
51 13 42
91 14 0
read ../data/cut_image/junior_college_pass/20110338.j

52 13 45
91 13 0
read ../data/cut_image/junior_college_pass/20110344.jpg
31 13 22
90 13 0
read ../data/cut_image/junior_college_pass/20110138.jpg
51 13 0
91 13 1
read ../data/cut_image/junior_college_pass/20110243.jpg
53 14 0
95 14 13
read ../data/cut_image/junior_college_pass/20110298.jpg
52 14 20
91 13 0
read ../data/cut_image/junior_college_pass/20110164.jpg
52 12 1
92 12 30
read ../data/cut_image/junior_college_pass/20140136.jpg
40 17 0
101 17 0
read ../data/cut_image/junior_college_pass/201100072.jpg
53 13 48
93 13 10
read ../data/cut_image/junior_college_pass/20110173.jpg
52 12 0
92 12 46
read ../data/cut_image/junior_college_pass/201100163.jpg
52 13 33
92 13 0
read ../data/cut_image/junior_college_pass/20110171.jpg
51 12 0
91 12 0
read ../data/cut_image/junior_college_pass/201100070.jpg
52 12 0
92 12 1
read ../data/cut_image/junior_college_pass/20110342.jpg
31 13 54
90 13 0
read ../data/cut_image/junior_college_pass/20110031.jpg
51 13 0
91 12 34
read ../data/cut_image/junior_col

read ../data/cut_image/undergraduate_pass/20110338.jpg
30 13 0
89 12 0
read ../data/cut_image/undergraduate_pass/20110329.jpg
52 12 0
92 12 0
read ../data/cut_image/undergraduate_pass/20110330.jpg
52 12 31
92 12 0
read ../data/cut_image/undergraduate_pass/20110327.jpg
52 12 0
91 12 38
read ../data/cut_image/undergraduate_pass/20110326.jpg
52 13 0
92 13 0
read ../data/cut_image/undergraduate_pass/20110347.jpg
30 14 36
89 14 0
read ../data/cut_image/undergraduate_pass/20140114.jpg
42 14 23
102 17 53
read ../data/cut_image/undergraduate_pass/20110085.jpg
52 12 1
91 11 33
read ../data/cut_image/undergraduate_pass/20110395.jpg
31 13 57
89 13 1
read ../data/cut_image/undergraduate_pass/20110082.jpg
52 12 0
91 12 0
read ../data/cut_image/undergraduate_pass/201100197.jpg
52 13 48
92 13 0
read ../data/cut_image/undergraduate_pass/20140016.jpg
43 19 31
104 19 0
read ../data/cut_image/undergraduate_pass/201100126.jpg
57 13 58
97 13 0
read ../data/cut_image/undergraduate_pass/201100150.jpg
55 13 3

26 13 37
86 13 0
read ../data/cut_image/undergraduate_pass/20110381.jpg
30 13 0
90 13 44
read ../data/cut_image/undergraduate_pass/20110331.jpg
53 13 36
92 12 0
read ../data/cut_image/undergraduate_pass/20110157.jpg
52 13 0
92 13 0
read ../data/cut_image/undergraduate_pass/201100205.jpg
56 13 0
95 13 0
read ../data/cut_image/undergraduate_pass/20110191.jpg
52 15 49
93 15 1
read ../data/cut_image/undergraduate_pass/201100161.jpg
55 14 51
94 13 1
read ../data/cut_image/undergraduate_pass/20140062.jpg
43 17 29
105 17 0
read ../data/cut_image/undergraduate_pass/20110051.jpg
51 12 0
91 12 29
read ../data/cut_image/undergraduate_pass/201100012.jpg
53 12 32
93 13 2
read ../data/cut_image/undergraduate_pass/20140300.jpg
43 16 31
103 15 0
read ../data/cut_image/undergraduate_pass/20110240.jpg
54 14 28
95 13 3
read ../data/cut_image/undergraduate_pass/201100027.jpg
51 13 0
91 13 47
read ../data/cut_image/undergraduate_pass/20110236.jpg
53 14 5
95 14 1
read ../data/cut_image/undergraduate_pass/20

read ../data/cut_image/graduate_pass/201100090.jpg
52 12 0
92 12 0
read ../data/cut_image/graduate_pass/20110339.jpg
30 13 0
90 13 0
read ../data/cut_image/graduate_pass/201100253.jpg
27 13 1
86 13 0
read ../data/cut_image/graduate_pass/20140311.jpg
43 16 0
103 16 35
read ../data/cut_image/graduate_pass/20140318.jpg
43 16 0
103 16 18
read ../data/cut_image/graduate_pass/20140299.jpg
42 16 0
102 16 24
read ../data/cut_image/graduate_pass/20110229.jpg
53 15 10
95 15 0
read ../data/cut_image/graduate_pass/20110112.jpg
51 13 0
91 13 0
read ../data/cut_image/graduate_pass/20110401.jpg
29 12 0
89 12 0
read ../data/cut_image/graduate_pass/20140326.jpg
43 15 2
103 15 29
read ../data/cut_image/graduate_pass/20110091.jpg
52 13 0
91 12 0
read ../data/cut_image/graduate_pass/201100309.jpg
42 17 0
101 17 19
read ../data/cut_image/graduate_pass/201100021.jpg
51 13 0
91 13 0
read ../data/cut_image/graduate_pass/20110148.jpg
52 13 29
92 13 0
read ../data/cut_image/graduate_pass/20130184.jpg
42 15 0
10

52 13 0
92 12 1
read ../data/cut_image/graduate_pass/2015002.jpg
42 16 0
103 16 7
read ../data/cut_image/graduate_pass/20110259.jpg
53 15 3
95 15 5
read ../data/cut_image/graduate_pass/201100249.jpg
26 13 0
85 13 0
read ../data/cut_image/graduate_pass/20110078.jpg
51 12 0
91 12 0
read ../data/cut_image/graduate_pass/201100220.jpg
26 13 0
86 13 0
read ../data/cut_image/graduate_pass/20110385.jpg
31 12 0
90 12 0
read ../data/cut_image/graduate_pass/20110322.jpg
54 13 2
92 25 3
read ../data/cut_image/graduate_pass/20140314.jpg
42 16 9
103 15 35
read ../data/cut_image/graduate_pass/201100250.jpg
31 13 0
90 13 0
read ../data/cut_image/graduate_pass/20110105.jpg
51 12 25
91 13 0
read ../data/cut_image/graduate_pass/20140327.jpg
43 15 0
104 15 39
read ../data/cut_image/graduate_pass/201100040.jpg
52 12 0
92 13 0
read ../data/cut_image/graduate_pass/20110100.jpg
51 12 0
90 12 0
read ../data/cut_image/graduate_pass/20110356.jpg
30 13 0
89 13 0
read ../data/cut_image/graduate_pass/201100134.jpg


In [4]:
graduate

['无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '是',
 '否',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '无',
 '无',
 '无',
 '无',
 '无',
 '是',
 '无',
 '无',
 '无',
 '是',
 '无',
 '是',
 '是',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '否',
 '否',
 '无',
 '无',
 '无',
 '否',
 '无',
 '否',
 '无',
 '是',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '无',
 '无',
 '无',
 '是',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '是',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '是',
 '无',
 '无',
 '是',
 '无',
 '无',
 '无',
 '否',
 '是',
 '无',
 '无',
 '否',
 '无',
 '无',
 '是',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '无',
 '否',
 '无',
 '无',
 '无',
 '无'

In [5]:
df['登记表编号'] = file_id
df['高中是否毕业'] = high_school
df['大专是否毕业'] = junior_college
df['本科是否毕业'] = undergraduate
df['研究生是否毕业'] = graduate
df['是否毕业（其它）'] = '无'

In [6]:
df

,登记表编号,高中是否毕业,大专是否毕业,本科是否毕业,研究生是否毕业,是否毕业（其它）
0,20110007,无,否,是,无,无
1,20110190,无,无,是,无,无
2,20110133,是,无,是,无,无
3,20110230,无,无,否,无,无
4,20110335,是,无,是,无,无
5,20110155,无,无,无,无,无
6,20140315,是,是,是,无,无
7,20110052,是,否,无,无,无
8,20110062,无,无,是,是,无
9,20140330,无,无,是,否,无


In [7]:
df.to_csv('../data/pass_or_not.csv',index=False)